# 숨고 데이터 분석

#### 숨고_비즈니스 데이터 분석_박기혁 <br>

* <b> 1. 데이터 수집[(크롤링 코드 링크)](https://github.com/bill7845/Jupyter_Repo/blob/master/jupyter_CODE/%EC%88%A8%EA%B3%A0_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D/%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb) </b>
* <b> 2. 데이터 전처리 </b>
* <b> 3. 데이터 분석(간단한 현황 분석 및 시각화) </b>

In [1]:
# 필요 라이브러리 import
import pandas as pd
import numpy as np
import json
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

# # Standard plotly 
import plotly
import plotly.graph_objs as go
import plotly.express as px
plotly.offline.init_notebook_mode(connected=True)

%matplotlib inline
sns.set()

#### 크롤링하여 저장해놓은 데이터를 불러온다

In [2]:
# read json

with open('유저별정보_200310_1',encoding='utf-8') as json_file:
    json_data_1 = json.load(json_file)
with open('유저별정보_200310_2',encoding='utf-8') as json_file:
    json_data_2 = json.load(json_file)
with open('유저별정보_200310_3',encoding='utf-8') as json_file:
    json_data_3 = json.load(json_file)

In [3]:
# json to DataFrame

df_1 = pd.DataFrame(json_data_1['data'])
df_2 = pd.DataFrame(json_data_2['data'])
df_3 = pd.DataFrame(json_data_3['data'])

#### 로드해온 총 3개의 데이터프레임을 하나로 이어붙인다.

In [4]:
df = pd.concat([df_1,df_2,df_3])

In [5]:
df.head(1)

,user_id,category,grade,hired_cnt,review_cnt,address,career,business_size,auth_business,auth_personal,payment
0,2458907,"[ 수학 과외 , 논술 과외 ]",0.0,신규고수,0,경기 안산시,1,1,None,본인 인증,"안전거래, 계좌이체, 현금결제 가능"


#### 유저별 메인카테고리 정보를 추가해준다.

In [6]:
with open('1st_user_list.txt', 'rb') as f:
    user = pickle.load(f)

answer = ['레슨','홈/리빙','이벤트','비즈니스','디자인/개발','건강/미용','알바','기타']

def main_category(x):
    result = []
    for  i in range(8):
        if x in user[i] : result.append(answer[i])
    
    return result

df['main_category'] = df['user_id'].apply(main_category)

# # change index
df.set_index('user_id',inplace=True)

In [7]:
df.head(3)

,category,grade,hired_cnt,review_cnt,address,career,business_size,auth_business,auth_personal,payment,main_category
user_id,,,,,,,,,,,
2458907,"[ 수학 과외 , 논술 과외 ]",0.0,신규고수,0,경기 안산시,1,1,None,본인 인증,"안전거래, 계좌이체, 현금결제 가능",[레슨]
1690162,"[ 중국어 과외 , TSC 과외 ]",5.0,1,6,경기도 성남시,20,1,사업자등록증 등록완료,본인 인증,카드결제 가능,[레슨]
1799917,"[ 집 인테리어 , 상업공간 인테리어 , 철거 , 아파트 인테리어 , 인테리...",0.0,신규고수,0,경기도 구리시,5,5,사업자등록증 등록완료,본인 인증,"계좌이체, 현금결제 가능","[홈/리빙, 디자인/개발]"


#### 만들어진 데이터프레임의 모양을 살펴본다.

In [8]:
df.shape 
# 총 14247 row와 13개의 컬럼으로 구성
# 14247명의 user 정보

(14247, 11)

<br>

## 1. 수월한 데이터 분석을 위한 데이터 다듬기

<br>

In [9]:
## 주소정보에서 시와 나머지 주소를 분리

df['city_address'] = df['address'].str.split(" ").str[1]
df['sub_address'] = df['address'].str.split(" ").str[2]

In [10]:
df['city_address'].unique()
## 지역명의 경우 통일되어 있지 않음 ex) 서울특별시, 서울시, 서울

array(['경기', '경기도', '서울', '인천', '전북', '제주', '부산', '인천광역시', '대구', '광주',
       '세종특별자치시', '경남', '제주특별자치도', '전라북도', '세종', '대구광역시', '대전광역시',
       '서울특별시', '대전', '부산광역시', '울산광역시', '충남', '충청남도', '충북', '충청북도', '울산',
       '광주광역시', '강원', '경상남도', '전라남도', '경북', '전남', '경상북도', '서울시', '강원도',
       '경남도'], dtype=object)

In [11]:
## 지역명을 통일
city_dict = {'서울':'서울시','서울특별자치시':'서울시','서울특별시':'서울시','서울시':'서울시',
            '강원':'강원도','강원도':'강원도',
            '대구광역시':'대구시','대구':'대구시','대구시':'대구시',
            '울산광역시':'울산시','울산':'울산시','울산시':'울산시',
            '경기':'경기도','경기도':'경기도',
            '충남':'충청남도','충청남도':'충청남도','충북':'충청북도','충청북도':'충청북도',
            '경남':'경상남도','경남도':'경상남도','경상남도':'경삼남도','경북':'경상북도','경상북도':'경상북도',
            '광주광역시':'광주시','광주':'광주시','광주시':'광주시',
            '제주특별자치도':'제주시','제주':'제주시','제주시':"제주시",
            '인천':'인천시','인천광역시':'인천시','인천시':'인천시',
            '대전':'대전시','대전광역시':'대전시','대전시':'대전시',
            '세종':'세종시','세종특별자치시':'세종시','세종시':'세종시',
            '전북':'전라북도','전라북도':'전라북도','전남':'전라남도','전라남도':'전라남도',
            '부산':'부산시','부산광역시':'부산시','부산시':'부산시'}

df['city_address'] = df['city_address'].map(city_dict)

In [12]:
## '신규고수'의 경우 hired_cnt에서 분리하여 '신규고수' 여부를 구분하는 컬럼을 추가해준다.
def new_user(x):
    if x == '신규고수' : return '신규고수'
    else : return np.nan
df['신규고수'] = df['hired_cnt'].apply(new_user)


df['business_size'] = df['business_size'].replace(np.nan,0)

In [13]:
## 정수형이여야할 컬럼들도 문자열로 되어있다.
df.select_dtypes('object').columns

Index(['category', 'grade', 'hired_cnt', 'review_cnt', 'address', 'career',
       'business_size', 'auth_business', 'auth_personal', 'payment',
       'main_category', 'city_address', 'sub_address', '신규고수'],
      dtype='object')

In [14]:
## 정수형으로 변경
df['grade'] = df['grade'].astype('float')
df['hired_cnt'] = df['hired_cnt'].apply(pd.to_numeric,errors = 'coerce') # 숫자형 제외한 나머지 nan으로
df['review_cnt'] = df['review_cnt'].astype('int')
df['career'] = df['career'].astype('int')
df['business_size'] = df['business_size'].astype('int')

In [15]:
# full address 삭제 
df.drop(['address'],axis=1,inplace=True)

In [16]:
# category 데이터 전처리
# [' 서비스명 '] => ['서비스명'] 앞뒤 공백제거
def del_strip(x):
    stripped_list = []
    for i in range(len(x)):
        stripped_list.append(x[i].strip())
    
    return stripped_list

df['category'] = df['category'].apply(del_strip)

In [17]:
df.head()

,category,grade,hired_cnt,review_cnt,career,business_size,auth_business,auth_personal,payment,main_category,city_address,sub_address,신규고수
user_id,,,,,,,,,,,,,
2458907,"[수학 과외, 논술 과외]",0.0,NaN,0,1,1,None,본인 인증,"안전거래, 계좌이체, 현금결제 가능",[레슨],경기도,안산시,신규고수
1690162,"[중국어 과외, TSC 과외]",5.0,1.0,6,20,1,사업자등록증 등록완료,본인 인증,카드결제 가능,[레슨],경기도,성남시,NaN
1799917,"[집 인테리어, 상업공간 인테리어, 철거, 아파트 인테리어, 인테리어 도면 (캐드/...",0.0,NaN,0,5,5,사업자등록증 등록완료,본인 인증,"계좌이체, 현금결제 가능","[홈/리빙, 디자인/개발]",경기도,구리시,신규고수
1811216,[중국어 과외],0.0,4.0,0,8,0,None,본인 인증,"안전거래, 계좌이체 가능",[레슨],서울시,광진구,NaN
1173470,"[일본어 (일어) 번역, 일본어(일어) 과외, 비즈니스 일본어]",5.0,13.0,18,3,1,None,본인 인증,"안전거래, 계좌이체, 현금결제 가능",[레슨],서울시,광진구,NaN


In [18]:
# 컬럼 순서변경
df = df[['main_category','category','city_address','sub_address','신규고수','hired_cnt','grade','review_cnt','career','business_size','auth_business','auth_personal','payment']]

<br>

## 2. 간단한 서비스 현황 파악

<br>

* <b> 크롤링 한 데이터를 숨고의 서비스 현황 파악 </b>
    * 1. 고수 고용횟수 현항 파악
    * 2. 고수별 도시정보 
    * 3. 서비스 카테고리 분포

> 1. `고용횟수 현황 파악`

In [19]:
df.hired_cnt.dropna().describe()

count    8401.000000
mean       13.697893
std        44.657809
min         0.000000
25%         1.000000
50%         4.000000
75%        11.000000
max      1749.000000
Name: hired_cnt, dtype: float64

> 신규고수를 제외하고 총 8401명 고수의 고용횟수를 살펴보면, 평균 13회 표준편차 44회로 고수별로 편차가 큰것을 확인할수 있다.

> 고용횟수의 편차가 매우 크므로 아래와 같이 1~300회 사이로 범위를 좁혀 시각화 해본다.

> 신규고수 제외 8401명의 고수들 중 7918명이 50회 이내의 고용횟수를 가지는것을 확인할 수 있다.

In [33]:
# df = px.data.tips()
# create the bins
counts, bins = np.histogram(df.hired_cnt.dropna(), bins=range(1, 300, 50))
bins = 0.5 * (bins[:-1] + bins[1:])

fig = px.bar(x=bins, y=counts, labels={'x':'고용횟수', 'y':'count'},title='고용횟수 분포',text=counts)
fig.show()

![img_1](https://user-images.githubusercontent.com/35517797/76479451-25f18280-644f-11ea-82cf-1310bc485f96.png)

> 2. `고수들의 도시정보 현황`

> 아래의 그래프로 보면, 서울,경기를 연고지로하는 고수가 70퍼센트 가까이 되는것을 확인할 수 있다.

In [39]:
city_count = pd.DataFrame(df.city_address.value_counts()).reset_index()
city_count['city'] = city_count['index']
city_count['count'] = city_count['city_address']

fig = px.pie(city_count, values='count', names='city',title='도시분포')
fig.show()

![img_2](https://user-images.githubusercontent.com/35517797/76479463-330e7180-644f-11ea-859f-53b5c4918067.png)

> 3. `서비스 카테고리별 현황`

> 고수별 카테고리 데이터의 경우 한명의 고수가 여러개의 카테고리를 가질 수 있다는점에 주의한다.

> 메인 카테고리 분류 기준은 8개의 메인 서비스 카테고리를 참고한다.

In [22]:
# 각 row에 list형식으로 된 메인카테고리 데이터를 핸들링하기 위한 함수 추가
mainCategory_dict = {}
mainCategory_dict = {'레슨':0,'홈/리빙':0,'이벤트':0,'비즈니스':0,'디자인/개발':0,'건강/미용':0,'알바':0,'기타':0}

def cnt_mainCategory(x):
    cnt = 0
    global mainCategory_dict
    mainCategory_list = ['레슨','홈/리빙','이벤트','비즈니스','디자인/개발','건강/미용','알바','기타']
    
    for i in x:
        if i == mainCategory_list[0] : mainCategory_dict[mainCategory_list[0]] +=1
        elif i == mainCategory_list[1] : mainCategory_dict[mainCategory_list[1]] +=1
        elif i == mainCategory_list[2] : mainCategory_dict[mainCategory_list[2]] +=1
        elif i == mainCategory_list[3] : mainCategory_dict[mainCategory_list[3]] +=1
        elif i == mainCategory_list[4] : mainCategory_dict[mainCategory_list[4]] +=1
        elif i == mainCategory_list[5] : mainCategory_dict[mainCategory_list[5]] +=1
        elif i == mainCategory_list[6] : mainCategory_dict[mainCategory_list[6]] +=1
        elif i == mainCategory_list[7] : mainCategory_dict[mainCategory_list[7]] +=1
#     print(mainCategory_dict)
#     mainCate_df = pd.DataFrame(mainCategory_dict.values(), columns=['count'], index=mainCategory_dict.keys())
    
#     return mainCate_df

df['main_category'].apply(cnt_mainCategory) # global 변수인 mainCategory_dict를 변경시키기 위해 실행

mainCate_df = pd.DataFrame(mainCategory_dict.values(), columns=['count'], index=mainCategory_dict.keys()).reset_index()

In [23]:
mainCate_df

,index,count
0,레슨,9334
1,홈/리빙,2720
2,이벤트,1614
3,비즈니스,680
4,디자인/개발,875
5,건강/미용,1240
6,알바,230
7,기타,778


> 전체 고수 중 53.4퍼센트의 고수가 '레슨' 서비스에 분포하고있으며 그 뒤로 홈리빙,이벤트.. 순으로 분포한다.

In [36]:
mainCate_df =mainCate_df.reset_index()

fig = px.pie(mainCate_df, values='count', names='index',title='메인 카테고리 분포 현황<고수 제공 서비스>')
fig.show()

![img_3](https://user-images.githubusercontent.com/35517797/76479509-5cc79880-644f-11ea-8f8b-d52e2ef60c07.png)

> 신규고수를 제외하고 실제로 고용을 경험한 고수들을 대상으로 살펴본다.

> 아래 그래프를 살펴보면, 마찬가지로 레슨,홈리빙 부문에 많은 고수들이 존재하는것을 확인할 수 있다.

In [25]:
mainCategory_dict = {}
mainCategory_dict = {'레슨':0,'홈/리빙':0,'이벤트':0,'비즈니스':0,'디자인/개발':0,'건강/미용':0,'알바':0,'기타':0}

df.dropna(subset=['hired_cnt'])['main_category'].apply(cnt_mainCategory) # global 변수인 mainCategory_dict를 변경시키기 위해 실행
mainCate_df = pd.DataFrame(mainCategory_dict.values(), columns=['count'], index=mainCategory_dict.keys()).reset_index()

In [37]:
mainCate_df =mainCate_df.reset_index()

fig = px.pie(mainCate_df, values='count', names='index',title='메인 카테고리 분포 현황<신규고수 제외>')
fig.show()

![img_4](https://user-images.githubusercontent.com/35517797/76479517-65b86a00-644f-11ea-99eb-01b895febd62.png)